In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy.io
import cv2
import random


In [2]:
# Parameters of data
img_size = 28
img_flat_size = img_size * img_size

num_label = 10 # 0 ~ 9

# Parameters of training
Learning_rate = 0.0005
epsilon = 1e-2

num_iter = 5100
batch_size = 100

validation_ratio = 0.1
gpu_fraction = 0.5

# Ensemble networks
networks = ['network1']

#keep_prob = tf.placeholder(tf.float32)
#batch_prob = tf.placeholder(tf.bool)

In [3]:
# Import MNIST dataset
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', validation_size=0)

# Dataset for train, test, validation
test_len = mnist.test.images.shape[0]
validation_len = int(test_len * validation_ratio)

train_x = mnist.train.images
test_x = mnist.test.images[validation_len : test_len, :]
validation_x = mnist.test.images[ : validation_len, :]

train_y_index = mnist.train.labels
test_y_index = mnist.test.labels[validation_len : test_len]
validation_y_index = mnist.test.labels[ : validation_len]

train_y = np.zeros([train_y_index.shape[0], num_label])
test_y = np.zeros([test_y_index.shape[0], num_label])
validation_y = np.zeros([validation_y_index.shape[0], num_label])

for i in range(train_y.shape[0]):
    train_y[i, train_y_index[i]] = 1

for i in range(test_y.shape[0]):
    test_y[i, test_y_index[i]] = 1
    
for i in range(validation_y.shape[0]):
    validation_y[i, validation_y_index[i]] = 1
    
print("\nTraining X shape: " + str(train_x.shape))
print("Testing X shape: " + str(test_x.shape))
print("Validation X shape: " + str(validation_x.shape))

print("\nTraining Y shape: " + str(train_y.shape))
print("Testing Y shape: " + str(test_y.shape))
print("Validation Y shape: " + str(validation_y.shape))

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.

Training X shape: (60000, 784)
Testing X shape: (9000, 784)
Validation X shape: (1000, 784)

Training Y shape: (60000, 10)
Testing Y shape: (9000, 10)
Validation Y shape: (1000, 10)


In [4]:
folders_notMNIST = os.listdir('C:/Users/Alchera/Desktop/jaeseung/code/uncertainty_project/notMNIST_small')

NotMNIST_x_list = []
NotMNIST_y_list = []

for idx, folder in enumerate(folders_notMNIST):
    files_notMNIST = os.listdir('./notMNIST_small/' + folder)
    
    for file in files_notMNIST:
        img_NotMNIST = cv2.imread('./notMNIST_small/' + folder + '/' + file, 0)
        NotMNIST_flat = np.reshape(img_NotMNIST, (img_flat_size))
        NotMNIST_x_list.append(NotMNIST_flat)
        
        label_temp = np.zeros([num_label])
        label_temp[idx] = 1
        
        NotMNIST_y_list.append(label_temp)
        
NotMNIST_x = np.stack(NotMNIST_x_list, axis = 0)
NotMNIST_y = np.stack(NotMNIST_y_list, axis = 0)

print("NotMNIST X shape: " + str(NotMNIST_x.shape))
print("NotMNIST Y shape: " + str(NotMNIST_y.shape))

NotMNIST X shape: (18674, 784)
NotMNIST Y shape: (18674, 10)


In [5]:
tf.reset_default_graph()

keep_prob = tf.placeholder(tf.float32)
batch_prob = tf.placeholder(tf.bool)
    
# Get Variables
def weight_variable(name, shape):
    return tf.get_variable(name, shape = shape, initializer = tf.contrib.layers.xavier_initializer())

def bias_variable(name, shape):
    return tf.get_variable(name, shape = shape, initializer = tf.contrib.layers.xavier_initializer())

# Get networks
def get_network(network_name):
    
    x_image = tf.placeholder(tf.float32, shape = [None, img_flat_size*1])
    y_label = tf.placeholder(tf.float32, shape = [None, num_label])
        
    n_hidden_1 = 200 # 1st layer number of features
    n_hidden_2 = 200 # 2nd layer number of features
    n_hidden_3 = 200 # 3rd layer
    
    with tf.variable_scope(network_name):
        
        # Densely connect layer variables
        w1 = weight_variable(network_name + '_w1', [img_flat_size, n_hidden_1])
        w2 = weight_variable(network_name + '_w2', [n_hidden_1,n_hidden_2])
        w3 = weight_variable(network_name + '_w3', [n_hidden_2,n_hidden_3])
        w4 = weight_variable(network_name + '_w4', [n_hidden_3,num_label])
        
        b1 = bias_variable(network_name + '_b1', [n_hidden_1])
        b2 = bias_variable(network_name + '_b2', [n_hidden_2])
        b3 = bias_variable(network_name + '_b3', [n_hidden_3])
        b4 = bias_variable(network_name + '_b4', [num_label])
        
        
    # Network
    L1 = tf.nn.relu(tf.matmul(x_image,w1) + b1)
    #L1 = batch_norm_layer(L1)
    #L1 = tf.contrib.layers.batch_norm(L1, center=True, scale=False, is_training=batch_prob)
    L1 = tf.nn.dropout(L1, keep_prob = keep_prob)
    
    L2 = tf.nn.relu(tf.matmul(L1,w2) + b2)
    #L2 = batch_norm_layer(L2)
    #L2 = tf.contrib.layers.batch_norm(L2, center=True, scale=False, is_training=batch_prob)
    L2 = tf.nn.dropout(L2, keep_prob = keep_prob)
    
    L3 = tf.nn.relu(tf.matmul(L2,w3) + b3)
    #L3 = batch_norm_layer(L3)
    #L3 = tf.contrib.layers.batch_norm(L3, center=True, scale=False, is_training=batch_prob)
    L3 = tf.nn.dropout(L3, keep_prob = keep_prob)
    
    logits = tf.matmul(L3,w4) + b4
    output = tf.nn.softmax(logits)
    
    #logits = tf.matmul(h3, w_fc3) + b_fc3


    
    # Brier Score 
    loss = tf.reduce_mean(tf.div(tf.reduce_sum(tf.square(tf.subtract(output, y_label)), axis = 1), num_label), axis = 0) 
   
    # Get trainable variables
    
    train_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, network_name)
    train_opt = tf.train.AdamOptimizer(Learning_rate).minimize(loss, var_list = train_vars)
    
    return x_image, y_label, output, loss, train_opt, train_vars


# Make batch data 
def making_batch(data_size, sample_size, data_x, data_y):
    # Making batches(testing)
    batch_idx = np.random.choice(data_size, sample_size)
    
    batch_x = np.zeros([sample_size, img_flat_size])
    batch_y = np.zeros([sample_size, num_label])
        
    for i in range(batch_idx.shape[0]):
        batch_x[i,:] = np.reshape(data_x[batch_idx[i], :], (img_flat_size))
        batch_y[i,:] = data_y[batch_idx[i], :] 
        
    return batch_x, batch_y

def making_batch_formlp(data_size, sample_size, data_x, data_y):
    # Making batches(testing)
    batch_idx = np.random.choice(data_size, sample_size)
    
    batch_x = np.zeros([sample_size, img_flat_size])
    batch_y = np.zeros([sample_size, num_label])
        
    for i in range(batch_idx.shape[0]):
        batch_x[i,:] = data_x[batch_idx[i], :]
        batch_y[i,:] = data_y[batch_idx[i], :]
        
    return batch_x, batch_y

# Calculate Accuracy
def get_accuracy(prediction, label):
    count_correct = 0
    for j in range(prediction.shape[0]):
        if np.argmax(label[j, :]) == np.argmax(prediction[j, :]):
            count_correct += 1.0
    
    acc = count_correct / label.shape[0]
    return acc

In [6]:
# Ensemble Networks
x_list = []
y_list = []
output_list = []
loss_list = []
train_list = []
train_var_list = []

# Train each ensemble network
for i in range(len(networks)):
    x_image, y_label, output, loss, train_opt, train_vars = get_network(networks[i])

    x_list.append(x_image)
    y_list.append(y_label)
    output_list.append(output)
    loss_list.append(loss)
    train_list.append(train_opt)
    train_var_list.append(train_vars)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Instructions for updating:
Use tf.cast instead.


In [7]:
# Create Session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = gpu_fraction

sess = tf.InteractiveSession(config=config)
sess.run(tf.global_variables_initializer())

In [8]:
# Initialize data for printing
loss_check     = np.zeros(len(networks))
acc_check      = np.zeros(len(networks))
acc_check_val = np.zeros(len(networks))
acc_check_val_final = 0

# Set parameters for printing and validating
num_print = 100
test_size = 10

train_data_num = train_x.shape[0]
val_data_num  = validation_x.shape[0]

for iter in range(num_iter):
    output_temp = np.zeros([test_size, num_label])

    # Making batches(validation)
    batch_x_val, batch_y_val = making_batch_formlp(val_data_num, test_size, validation_x, validation_y)
        
    for i in range(len(networks)):
        # Making batches(training)
        batch_x, batch_y = making_batch_formlp(train_data_num, batch_size, train_x, train_y)
       
        # Training
        _, loss, prob = sess.run([train_list[i], loss_list[i], output_list[i]], 
                                 feed_dict = {x_list[i]: batch_x, y_list[i]: batch_y,keep_prob:0.9})
        
        # Validation
        loss_test, prob_test = sess.run([loss_list[i], output_list[i]], 
                                         feed_dict = {x_list[i]: batch_x_val, y_list[i]: batch_y_val,keep_prob:0.9})
        
        # Add test prediction for get final prediction
        output_temp += prob_test
        # Calculate Accuracy (Training)
        acc_training = get_accuracy(prob, batch_y)
 
        # Calculate Accuracy (validating)
        acc_validating = get_accuracy(prob_test, batch_y_val)
                
        # Get accuracy and loss for each network
        acc_check[i] += acc_training
        acc_check_val[i] += acc_validating
        loss_check[i] += loss
    
    # Get final test prediction
    prob_val_final = output_temp / len(networks)
    # Calculate Accuracy (Validating final)
    acc_validating_final = get_accuracy(prob_val_final, batch_y_val)
    acc_check_val_final += acc_validating_final
    
    if iter % num_print == 0 and iter != 0:
        print(('-------------------------') + ' Iteration: ' + str(iter) + ' -------------------------')
        print('Average Loss(Brier score): ' + str(loss_check / num_print))
        print('Training Accuracy: ' + str(acc_check / num_print))
        print('Validating Accuracy: ' + str(acc_check_val / num_print))
        print('Final Validating Accuracy: ' + str(acc_check_val_final / num_print))
        print('\n')
        
        loss_check = np.zeros(len(networks))
        acc_check = np.zeros(len(networks))
        acc_check_val = np.zeros(len(networks))
        acc_check_val_final = 0

------------------------- Iteration: 100 -------------------------
Average Loss(Brier score): [0.04197083]
Training Accuracy: [0.7068]
Validating Accuracy: [0.68]
Final Validating Accuracy: 0.6799999999999998


------------------------- Iteration: 200 -------------------------
Average Loss(Brier score): [0.01600927]
Training Accuracy: [0.8953]
Validating Accuracy: [0.895]
Final Validating Accuracy: 0.8949999999999997


------------------------- Iteration: 300 -------------------------
Average Loss(Brier score): [0.01250492]
Training Accuracy: [0.9195]
Validating Accuracy: [0.915]
Final Validating Accuracy: 0.9150000000000003


------------------------- Iteration: 400 -------------------------
Average Loss(Brier score): [0.01055245]
Training Accuracy: [0.9318]
Validating Accuracy: [0.907]
Final Validating Accuracy: 0.9070000000000001


------------------------- Iteration: 500 -------------------------
Average Loss(Brier score): [0.01024135]
Training Accuracy: [0.9345]
Validating Accurac

------------------------- Iteration: 4000 -------------------------
Average Loss(Brier score): [0.00270948]
Training Accuracy: [0.983]
Validating Accuracy: [0.969]
Final Validating Accuracy: 0.9689999999999999


------------------------- Iteration: 4100 -------------------------
Average Loss(Brier score): [0.0021517]
Training Accuracy: [0.9866]
Validating Accuracy: [0.967]
Final Validating Accuracy: 0.9670000000000003


------------------------- Iteration: 4200 -------------------------
Average Loss(Brier score): [0.00239755]
Training Accuracy: [0.9851]
Validating Accuracy: [0.966]
Final Validating Accuracy: 0.9660000000000001


------------------------- Iteration: 4300 -------------------------
Average Loss(Brier score): [0.0025118]
Training Accuracy: [0.9839]
Validating Accuracy: [0.973]
Final Validating Accuracy: 0.9730000000000001


------------------------- Iteration: 4400 -------------------------
Average Loss(Brier score): [0.0021728]
Training Accuracy: [0.9863]
Validating Accur

In [17]:
#testset 중 9000개의 인덱스 추출
MNIST_sample_index = np.random.choice(test_x.shape[0],9000)
print(MNIST_sample_index)
Macc_final_final_en = 0
Macc_final_final_on = 0

output_sample     = []
output_sample_one = []

m_en = []
m_one = []

for i, index in enumerate(MNIST_sample_index):
    img_MNIST = np.reshape(test_x[index], (img_flat_size))
    output_test1 = np.zeros([1, num_label])
    output_test2 = np.zeros([1, num_label])
    output_test3 = np.zeros([1, num_label])
    output_test4 = np.zeros([1, num_label])
    output_test5 = np.zeros([1, num_label])



    #각 네트위크별로 prob를 구한다음 계속 더함
    x_temp = np.reshape(test_x[index, :], (1, img_flat_size))
    y_temp = np.reshape(test_y[index, :], (1, num_label))

    loss_temp, prob_temp1 = sess.run([loss_list[0], output_list[0]], 
                                        feed_dict = {x_list[0]: x_temp, y_list[0]: y_temp, keep_prob:0.9})     
    loss_temp, prob_temp2 = sess.run([loss_list[0], output_list[0]], 
                                        feed_dict = {x_list[0]: x_temp, y_list[0]: y_temp, keep_prob:0.9})
    loss_temp, prob_temp3 = sess.run([loss_list[0], output_list[0]], 
                                        feed_dict = {x_list[0]: x_temp, y_list[0]: y_temp, keep_prob:0.9})
    loss_temp, prob_temp4 = sess.run([loss_list[0], output_list[0]], 
                                        feed_dict = {x_list[0]: x_temp, y_list[0]: y_temp, keep_prob:0.9})
    loss_temp, prob_temp5 = sess.run([loss_list[0], output_list[0]], 
                                        feed_dict = {x_list[0]: x_temp, y_list[0]: y_temp, keep_prob:0.9})

    # Add test prediction for get final prediction
    output_test1 += prob_temp1
    output_test2 += prob_temp2
    output_test3 += prob_temp3
    output_test4 += prob_temp4
    output_test5 += prob_temp5

        
    # Get final test prediction
    prob_temp_final = (output_test1 + output_test2 + output_test3 + output_test4 + output_test5)/5
    #print('prob_temp_final: ' + str(prob_temp_final))
    prob_temp_one   = prob_temp1
    
    # Get accuracy
    acc_final_check_en_m = get_accuracy(prob_temp_final, y_temp)
    Macc_final_final_en += acc_final_check_en_m
    acc_final_check_on_m = get_accuracy(prob_temp_one, y_temp)
    Macc_final_final_on += acc_final_check_on_m
    
    m_en.append(acc_final_check_en_m)
    m_one.append(acc_final_check_on_m)
    output_sample.append(prob_temp_final)
    output_sample_one.append(prob_temp_one)

[8768 1290  916 ... 8500  412 6025]


In [25]:
print("====================== Dropout Result ======================")
array_ensemble_MNIST = np.zeros([9000])
conf = 0.2
count_ex_m_en = 0
correction_m_en = 0
for i in range(len(output_sample)):
    
    idx_sample = np.argmax(output_sample[i])
    max_prob = output_sample[i][0, idx_sample]
    array_ensemble_MNIST[i] = max_prob
    
    if max_prob >= conf:
        
        count_ex_m_en += 1
        correction_m_en += m_en[i]
        
    #print(str(i+1) + 'th sample: label = ' + str(idx_sample) + ', Confidence = ' + str(max_prob) + ', Correction = ' + str(m_en[i]))
print("Accuracy: " + str(Macc_final_final_en/9000) + " Satisfying MNIST: " + str(correction_m_en))
print("Total: " + str(count_ex_m_en))

====================== Dropout Result ======================
Accuracy: 0.9793333333333333 Satisfying MNIST: 8814.0
Total: 9000


In [11]:
print("\n====================== SingleNet Result ======================")
array_single_MNIST = np.zeros([9000])
count_ex_m_on = 0
correction_m_on = 0
for i in range(len(output_sample_one)):
    
    idx_sample = np.argmax(output_sample_one[i])
    max_prob = output_sample_one[i][0, idx_sample]
    array_single_MNIST[i] = max_prob
    
    if max_prob >= conf:
        count_ex_m_on += 1
        correction_m_on += m_one[i]
        
    print(str(i+1) + 'th sample: label = ' + str(idx_sample) + ', Confidence = ' + str(max_prob) + ', Correction = ' + str(m_one[i]))
print("Accuracy: " + str(Macc_final_final_on/9000) + " Condition: " + str(correction_m_on))
print("Total: " +str(count_ex_m_on))


====================== SingleNet Result ======================
1th sample: label = 5, Confidence = 0.9962807, Correction = 1.0
2th sample: label = 2, Confidence = 0.9961398, Correction = 1.0
3th sample: label = 1, Confidence = 0.9984251, Correction = 1.0
4th sample: label = 4, Confidence = 0.9999999, Correction = 1.0
5th sample: label = 4, Confidence = 1.0, Correction = 1.0
6th sample: label = 1, Confidence = 0.9980774, Correction = 1.0
7th sample: label = 0, Confidence = 0.9999999, Correction = 1.0
8th sample: label = 8, Confidence = 0.8926991, Correction = 1.0
9th sample: label = 4, Confidence = 0.9999999, Correction = 1.0
10th sample: label = 0, Confidence = 0.99999976, Correction = 1.0
11th sample: label = 0, Confidence = 0.9999809, Correction = 1.0
12th sample: label = 8, Confidence = 0.9996381, Correction = 1.0
13th sample: label = 8, Confidence = 0.92318165, Correction = 1.0
14th sample: label = 1, Confidence = 0.9998951, Correction = 1.0
15th sample: label = 8, Confidence = 0.

2568th sample: label = 6, Confidence = 1.0, Correction = 1.0
2569th sample: label = 7, Confidence = 0.9999933, Correction = 1.0
2570th sample: label = 4, Confidence = 0.9999665, Correction = 1.0
2571th sample: label = 3, Confidence = 0.68847036, Correction = 1.0
2572th sample: label = 1, Confidence = 0.9993304, Correction = 1.0
2573th sample: label = 4, Confidence = 0.9998654, Correction = 1.0
2574th sample: label = 2, Confidence = 0.9999994, Correction = 1.0
2575th sample: label = 0, Confidence = 0.95374674, Correction = 1.0
2576th sample: label = 4, Confidence = 1.0, Correction = 1.0
2577th sample: label = 5, Confidence = 0.98717, Correction = 1.0
2578th sample: label = 1, Confidence = 0.99704546, Correction = 1.0
2579th sample: label = 9, Confidence = 0.998944, Correction = 1.0
2580th sample: label = 9, Confidence = 0.979215, Correction = 1.0
2581th sample: label = 0, Confidence = 0.99994564, Correction = 1.0
2582th sample: label = 4, Confidence = 1.0, Correction = 1.0
2583th sample

5068th sample: label = 2, Confidence = 0.6019911, Correction = 0.0
5069th sample: label = 0, Confidence = 1.0, Correction = 1.0
5070th sample: label = 6, Confidence = 1.0, Correction = 1.0
5071th sample: label = 0, Confidence = 0.99978644, Correction = 1.0
5072th sample: label = 2, Confidence = 0.9999517, Correction = 1.0
5073th sample: label = 6, Confidence = 0.99997497, Correction = 1.0
5074th sample: label = 1, Confidence = 0.99992156, Correction = 1.0
5075th sample: label = 5, Confidence = 0.99999976, Correction = 1.0
5076th sample: label = 5, Confidence = 0.9999968, Correction = 1.0
5077th sample: label = 1, Confidence = 0.99998915, Correction = 1.0
5078th sample: label = 6, Confidence = 0.9999989, Correction = 1.0
5079th sample: label = 6, Confidence = 0.99997103, Correction = 1.0
5080th sample: label = 7, Confidence = 0.9999738, Correction = 1.0
5081th sample: label = 5, Confidence = 1.0, Correction = 1.0
5082th sample: label = 6, Confidence = 0.99187213, Correction = 1.0
5083th

8067th sample: label = 9, Confidence = 0.96901804, Correction = 1.0
8068th sample: label = 6, Confidence = 0.99341923, Correction = 1.0
8069th sample: label = 9, Confidence = 0.99652004, Correction = 1.0
8070th sample: label = 1, Confidence = 0.99995315, Correction = 1.0
8071th sample: label = 0, Confidence = 1.0, Correction = 1.0
8072th sample: label = 1, Confidence = 0.99987745, Correction = 1.0
8073th sample: label = 0, Confidence = 0.99999833, Correction = 1.0
8074th sample: label = 0, Confidence = 1.0, Correction = 1.0
8075th sample: label = 8, Confidence = 0.99980634, Correction = 1.0
8076th sample: label = 3, Confidence = 0.99703074, Correction = 1.0
8077th sample: label = 5, Confidence = 0.99999774, Correction = 1.0
8078th sample: label = 3, Confidence = 0.474111, Correction = 0.0
8079th sample: label = 6, Confidence = 0.99999976, Correction = 1.0
8080th sample: label = 2, Confidence = 0.99999416, Correction = 1.0
8081th sample: label = 2, Confidence = 0.9997758, Correction = 1

In [27]:
NotMNIST_sample_index = np.random.choice(NotMNIST_x.shape[0], 9000)

Nacc_final_final_en = 0
Nacc_final_final_on = 0

output_sample     = []
output_sample_one = []

n_en = []
n_one = []

for i, index in enumerate(NotMNIST_sample_index):    
    output_test1 = np.zeros([1, num_label])
    output_test2 = np.zeros([1, num_label])
    output_test3 = np.zeros([1, num_label])
    output_test4 = np.zeros([1, num_label])
    output_test5 = np.zeros([1, num_label])

    
    x_temp = np.reshape(NotMNIST_x[index, :], (1, img_flat_size))
    y_temp = np.reshape(NotMNIST_y[index, :], (1, num_label))
        
    loss_temp, prob_temp1 = sess.run([loss_list[0], output_list[0]], 
                                        feed_dict = {x_list[0]: x_temp, y_list[0]: y_temp, keep_prob:0.9})      
    loss_temp, prob_temp2 = sess.run([loss_list[0], output_list[0]], 
                                        feed_dict = {x_list[0]: x_temp, y_list[0]: y_temp, keep_prob:0.9})
    loss_temp, prob_temp3 = sess.run([loss_list[0], output_list[0]], 
                                        feed_dict = {x_list[0]: x_temp, y_list[0]: y_temp, keep_prob:0.9})
    loss_temp, prob_temp4 = sess.run([loss_list[0], output_list[0]], 
                                        feed_dict = {x_list[0]: x_temp, y_list[0]: y_temp, keep_prob:0.9})
    loss_temp, prob_temp5 = sess.run([loss_list[0], output_list[0]], 
                                        feed_dict = {x_list[0]: x_temp, y_list[0]: y_temp, keep_prob:0.9})

    # Add test prediction for get final prediction
    output_test1 += prob_temp1
    output_test2 += prob_temp2
    output_test3 += prob_temp3
    output_test4 += prob_temp4
    output_test5 += prob_temp5

        #print('output_test: ' + str(output_test))
        
    # Get final test prediction
    prob_temp_final = (output_test1 +output_test2 + output_test3 + output_test4 + output_test5)/5
    prob_temp_one   = prob_temp1
    
    acc_final_check_en_n = 0
    Nacc_final_final_en += acc_final_check_en_n
    acc_final_check_on_n = 0
    Nacc_final_final_on += acc_final_check_on_n

    n_en.append(acc_final_check_en_n)
    n_one.append(acc_final_check_on_n)
    
    output_sample.append(prob_temp_final)
    output_sample_one.append(prob_temp_one)

In [34]:
print("====================== Dropout Result ======================")
array_ensemble_NotMNIST = np.zeros([9000])
count_ex_n_en = 0
tot_prob_en = 0
conf = 0.2
for i in range(len(output_sample)):   
    idx_sample = np.argmax(output_sample[i])
    max_prob = output_sample[i][0, idx_sample]
    array_ensemble_NotMNIST[i] = max_prob
    tot_prob_en += max_prob
    
    if max_prob >= conf:
        count_ex_n_en += 1
    #print(str(i+1) + 'th sample: label = ' + str(idx_sample) + ', Confidence = ' + str(max_prob))
            
print("Accuracy: " + str(Nacc_final_final_en/9000))
print('Avg confidence: ' + str(tot_prob_en/9000))
print('Satisfying Ensemble NotMNIST: ' + str(count_ex_n_en))
total_count_en = count_ex_m_en + count_ex_n_en
acc_m_en = (correction_m_en/total_count_en)*100
print('Total accuracy: ' + str(acc_m_en))

====================== Dropout Result ======================
Accuracy: 0.0
Avg confidence: 0.834259051932014
Satisfying Ensemble NotMNIST: 9000
Total accuracy: 48.96666666666666


In [14]:
print("\n====================== SingleNet Result ======================")
array_single_NotMNIST = np.zeros([10800])
count_ex_n_one = 0
tot_prob_on = 0
for i in range(len(output_sample_one)):    
    idx_sample = np.argmax(output_sample_one[i])
    max_prob = output_sample_one[i][0, idx_sample]
    array_single_NotMNIST[i] = max_prob
    tot_prob_on += max_prob
    
    if max_prob >= conf:
        count_ex_n_one += 1
        
    print(str(i+1) + 'th sample: label = ' + str(idx_sample) + ', Confidence = ' + str(max_prob))
print("Accuracy: " + str(Nacc_final_final_on/9000))
print("Avg confidence : " + str(tot_prob_on/9000))
print('Satisfying Single NotMNIST: ' + str(count_ex_n_one))


====================== SingleNet Result ======================
1th sample: label = 7, Confidence = 1.0
2th sample: label = 5, Confidence = 1.0
3th sample: label = 8, Confidence = 1.0
4th sample: label = 3, Confidence = 1.0
5th sample: label = 7, Confidence = 1.0
6th sample: label = 8, Confidence = 1.0
7th sample: label = 5, Confidence = 1.0
8th sample: label = 7, Confidence = 1.0
9th sample: label = 5, Confidence = 1.0
10th sample: label = 7, Confidence = 1.0
11th sample: label = 3, Confidence = 1.0
12th sample: label = 2, Confidence = 1.0
13th sample: label = 3, Confidence = 1.0
14th sample: label = 6, Confidence = 1.0
15th sample: label = 5, Confidence = 1.0
16th sample: label = 1, Confidence = 1.0
17th sample: label = 3, Confidence = 0.99967504
18th sample: label = 7, Confidence = 1.0
19th sample: label = 6, Confidence = 1.0
20th sample: label = 5, Confidence = 1.0
21th sample: label = 6, Confidence = 1.0
22th sample: label = 3, Confidence = 1.0
23th sample: label = 6, Confidence =

2541th sample: label = 6, Confidence = 1.0
2542th sample: label = 5, Confidence = 1.0
2543th sample: label = 7, Confidence = 1.0
2544th sample: label = 1, Confidence = 1.0
2545th sample: label = 7, Confidence = 1.0
2546th sample: label = 7, Confidence = 1.0
2547th sample: label = 7, Confidence = 1.0
2548th sample: label = 3, Confidence = 1.0
2549th sample: label = 1, Confidence = 1.0
2550th sample: label = 9, Confidence = 1.0
2551th sample: label = 1, Confidence = 1.0
2552th sample: label = 0, Confidence = 1.0
2553th sample: label = 4, Confidence = 1.0
2554th sample: label = 2, Confidence = 1.0
2555th sample: label = 7, Confidence = 1.0
2556th sample: label = 2, Confidence = 1.0
2557th sample: label = 8, Confidence = 1.0
2558th sample: label = 3, Confidence = 1.0
2559th sample: label = 5, Confidence = 1.0
2560th sample: label = 8, Confidence = 1.0
2561th sample: label = 2, Confidence = 1.0
2562th sample: label = 5, Confidence = 1.0
2563th sample: label = 5, Confidence = 1.0
2564th samp

2832th sample: label = 6, Confidence = 1.0
2833th sample: label = 5, Confidence = 1.0
2834th sample: label = 7, Confidence = 1.0
2835th sample: label = 5, Confidence = 1.0
2836th sample: label = 9, Confidence = 1.0
2837th sample: label = 7, Confidence = 1.0
2838th sample: label = 5, Confidence = 1.0
2839th sample: label = 2, Confidence = 1.0
2840th sample: label = 5, Confidence = 1.0
2841th sample: label = 5, Confidence = 1.0
2842th sample: label = 0, Confidence = 1.0
2843th sample: label = 5, Confidence = 1.0
2844th sample: label = 9, Confidence = 1.0
2845th sample: label = 2, Confidence = 1.0
2846th sample: label = 5, Confidence = 1.0
2847th sample: label = 8, Confidence = 1.0
2848th sample: label = 5, Confidence = 1.0
2849th sample: label = 9, Confidence = 1.0
2850th sample: label = 0, Confidence = 1.0
2851th sample: label = 8, Confidence = 1.0
2852th sample: label = 6, Confidence = 1.0
2853th sample: label = 7, Confidence = 1.0
2854th sample: label = 3, Confidence = 1.0
2855th samp

5040th sample: label = 5, Confidence = 1.0
5041th sample: label = 6, Confidence = 1.0
5042th sample: label = 7, Confidence = 1.0
5043th sample: label = 5, Confidence = 1.0
5044th sample: label = 6, Confidence = 1.0
5045th sample: label = 5, Confidence = 1.0
5046th sample: label = 6, Confidence = 1.0
5047th sample: label = 5, Confidence = 1.0
5048th sample: label = 7, Confidence = 1.0
5049th sample: label = 9, Confidence = 1.0
5050th sample: label = 5, Confidence = 1.0
5051th sample: label = 7, Confidence = 1.0
5052th sample: label = 5, Confidence = 1.0
5053th sample: label = 9, Confidence = 1.0
5054th sample: label = 3, Confidence = 1.0
5055th sample: label = 6, Confidence = 1.0
5056th sample: label = 5, Confidence = 1.0
5057th sample: label = 1, Confidence = 1.0
5058th sample: label = 6, Confidence = 1.0
5059th sample: label = 5, Confidence = 1.0
5060th sample: label = 7, Confidence = 1.0
5061th sample: label = 5, Confidence = 1.0
5062th sample: label = 4, Confidence = 1.0
5063th samp

5515th sample: label = 5, Confidence = 1.0
5516th sample: label = 5, Confidence = 1.0
5517th sample: label = 5, Confidence = 1.0
5518th sample: label = 5, Confidence = 1.0
5519th sample: label = 2, Confidence = 1.0
5520th sample: label = 7, Confidence = 1.0
5521th sample: label = 2, Confidence = 1.0
5522th sample: label = 3, Confidence = 1.0
5523th sample: label = 2, Confidence = 1.0
5524th sample: label = 7, Confidence = 1.0
5525th sample: label = 3, Confidence = 1.0
5526th sample: label = 3, Confidence = 1.0
5527th sample: label = 6, Confidence = 1.0
5528th sample: label = 5, Confidence = 1.0
5529th sample: label = 1, Confidence = 1.0
5530th sample: label = 1, Confidence = 1.0
5531th sample: label = 5, Confidence = 1.0
5532th sample: label = 5, Confidence = 1.0
5533th sample: label = 7, Confidence = 1.0
5534th sample: label = 9, Confidence = 1.0
5535th sample: label = 3, Confidence = 1.0
5536th sample: label = 5, Confidence = 1.0
5537th sample: label = 5, Confidence = 1.0
5538th samp

7954th sample: label = 7, Confidence = 1.0
7955th sample: label = 9, Confidence = 1.0
7956th sample: label = 4, Confidence = 1.0
7957th sample: label = 5, Confidence = 1.0
7958th sample: label = 3, Confidence = 1.0
7959th sample: label = 1, Confidence = 1.0
7960th sample: label = 1, Confidence = 1.0
7961th sample: label = 5, Confidence = 1.0
7962th sample: label = 5, Confidence = 1.0
7963th sample: label = 7, Confidence = 1.0
7964th sample: label = 2, Confidence = 1.0
7965th sample: label = 7, Confidence = 1.0
7966th sample: label = 5, Confidence = 1.0
7967th sample: label = 3, Confidence = 1.0
7968th sample: label = 5, Confidence = 1.0
7969th sample: label = 0, Confidence = 1.0
7970th sample: label = 6, Confidence = 1.0
7971th sample: label = 6, Confidence = 1.0
7972th sample: label = 3, Confidence = 1.0
7973th sample: label = 7, Confidence = 1.0
7974th sample: label = 8, Confidence = 1.0
7975th sample: label = 4, Confidence = 1.0
7976th sample: label = 3, Confidence = 1.0
7977th samp